## Calculate signal-to-noise ratio for SAT trend pattern during 1958-2022

In [2]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

<jemalloc>: arena 0 background thread creation failed (11)


In [3]:
data_hist_ssp245_MPI_ESM  = '/work/mh0033/m301036/LSAT/CMIP6-MPI-M-LR/MergeDataOut/tas_Amon_1850-2022_*.nc'
ds = xr.open_mfdataset(data_hist_ssp245_MPI_ESM, combine = 'nested', concat_dim = 'run')
ds

<xarray.Dataset>
Dimensions:    (time: 2076, run: 30, bnds: 2, lon: 180, lat: 90)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon        (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * lat        (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
    height     float64 2.0
Dimensions without coordinates: run, bnds
Data variables:
    time_bnds  (run, time, bnds) datetime64[ns] dask.array<chunksize=(1, 2076, 2), meta=np.ndarray>
    tas        (run, time, lat, lon) float32 dask.array<chunksize=(1, 2076, 90, 180), meta=np.ndarray>
Attributes: (12/49)
    CDI:                    Climate Data Interface version 2.0.3 (https://mpi...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology, Hamburg 201...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/6b679cba-17b8-45eb-90dc-23d170c1998c
    CDO:                    Climate Data Operators version 2.0.3 (https://mpi...

In [4]:
land_sea_mask=xr.open_dataset('/work/mh0033/m301036/LSAT/CMIP6-MPI-M-LR/GR15_lsm_regrid.nc')
mask_data = land_sea_mask['var1']
mask_data
# Align the time coordinates between the mask dataset and the original dataset
time_values = pd.to_datetime(mask_data['time'].values, format='mixed', dayfirst=True)
mask_data['time'] = time_values

# Align the time coordinates between the mask dataset and the original dataset
mask_data = mask_data.reindex(time=ds['time'], method='nearest')

# Apply the land-sea mask to the original dataset
masked_tas = ds.where(mask_data == 1, drop=False)
masked_tas

<xarray.Dataset>
Dimensions:    (run: 30, time: 2076, bnds: 2, lat: 90, lon: 180)
Coordinates:
  * time       (time) datetime64[ns] 1850-01-16T12:00:00 ... 2022-12-16T12:00:00
  * lon        (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * lat        (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
    height     float64 2.0
Dimensions without coordinates: run, bnds
Data variables:
    time_bnds  (run, time, bnds, lat, lon) datetime64[ns] dask.array<chunksize=(1, 2076, 2, 90, 180), meta=np.ndarray>
    tas        (run, time, lat, lon) float32 dask.array<chunksize=(1, 2076, 90, 180), meta=np.ndarray>
Attributes: (12/49)
    CDI:                    Climate Data Interface version 2.0.3 (https://mpi...
    source:                 MPI-ESM1.2-LR (2017): \naerosol: none, prescribed...
    institution:            Max Planck Institute for Meteorology, Hamburg 201...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    ...                     ...
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by MPI-M is licensed un...
    cmor_version:           3.5.0
    tracking_id:            hdl:21.14100/6b679cba-17b8-45eb-90dc-23d170c1998c
    CDO:                    Climate Data Operators version 2.0.3 (https://mpi...

In [5]:
tas = masked_tas['tas'].loc[:,'1958-01-01':'2022-12-31',0:90,:]
tas = tas - 273.15
print(tas.min().values)
tas_climatology = tas.groupby('time.month').mean(dim='time')
tas_climatology
tas_ano = tas.groupby('time.month') - tas_climatology
tas_ano_ds = tas_ano.to_dataset()
tas_ano_ds

RuntimeError: can't start new thread

In [ ]:
seasons = ['JJA', 'DJF', 'MAM', 'SON']
season_means = {}

for season in seasons:
    if season == 'JJA':
        months = [6,7,8]
    elif season == 'DJF':
        months =[12,1,2]
    elif season == 'MAM':
        months = [3,4,5]
    elif season == 'SON':
        months = [9,10,11]

    season_months = tas_ano_ds.sel(time=tas.time.dt.month.isin(months))
    
    # Calculate the seasonal mean SAT anomalies
    season_mean_anomalies = season_months.groupby('time.year').mean('time')
    
    # Store the seasonal mean in the dictionary
    season_means[season] = season_mean_anomalies['tas']

# Access the multiyear JJA mean SAT anomalies
    
tas_ano_ds['JJA'] = season_means['JJA']
tas_ano_ds['DJF'] = season_means['DJF']
tas_ano_ds['MAM'] = season_means['MAM']
tas_ano_ds['SON'] = season_means['SON']

tas_ano_ds

: 

In [ ]:
import scipy.stats as stats
from scipy.stats import linregress

def calc_trend(data):
    slope, intercept, r_value, p_value, std_err = linregress(np.arange(len(data)), data)  
    return slope, p_value

: 

In [ ]:
tas_ano_ds['slope_DJF'], tas_ano_ds['p_value_DJF'] = xr.apply_ufunc(calc_trend, tas_ano_ds['DJF'].chunk(dict(run=-1, year=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
tas_ano_ds['slope_DJF'].attrs['units'] = '°C/65yrs'
tas_ano_ds['p_value_DJF'].attrs['units'] = 'p_value'

: 

In [ ]:
tas_ano_ds['slope_MAM'], tas_ano_ds['p_value_MAM'] = xr.apply_ufunc(calc_trend, tas_ano_ds['MAM'].chunk(dict(run=-1, year=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
tas_ano_ds['slope_MAM'].attrs['units'] = '°C/65yrs'
tas_ano_ds['p_value_MAM'].attrs['units'] = 'p_value'

: 

In [ ]:
tas_ano_ds['slope_JJA'], tas_ano_ds['p_value_JJA'] = xr.apply_ufunc(calc_trend, tas_ano_ds['JJA'].chunk(dict(run=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
tas_ano_ds['slope_JJA'].attrs['units'] = '°C/65yrs'
tas_ano_ds['p_value_JJA'].attrs['units'] = 'p_value'

: 

In [ ]:
tas_ano_ds['slope_SON'], tas_ano_ds['p_value_SON'] = xr.apply_ufunc(calc_trend, tas_ano_ds['SON'].chunk(dict(run=-1, year=-1)), input_core_dims=[['year']], output_core_dims=[[],[]], vectorize=True, dask='parallelized', output_dtypes=[float,float], dask_gufunc_kwargs={'allow_rechunk': True})
tas_ano_ds['slope_SON'].attrs['units'] = '°C/65yrs'
tas_ano_ds['p_value_SON'].attrs['units'] = 'p_value'

: 

In [ ]:
tas_ano_ds = tas_ano_ds.compute()

: 

In [ ]:
tas_ano_ds

: 

## Calculate the DJF SAT trend pattern's variance among the 30 runs

In [ ]:
#Calculate the stadard deviation of the SAT signal to noise ratio for each grid cell
#The signal is the SAT trend from 1958-2022

#Import modules
from scipy import stats

#Read in the SAT trend data
DJF_trend = tas_ano_ds['slope_DJF']
MAM_trend = tas_ano_ds['slope_MAM']
JJA_trend = tas_ano_ds['slope_JJA']
SON_trend = tas_ano_ds['slope_SON']

#Calculate the standard deviation of the SAT trend
DJF_trend_std = DJF_trend.std(dim='run')
MAM_trend_std = MAM_trend.std(dim='run')
JJA_trend_std = JJA_trend.std(dim='run')
SON_trend_std = SON_trend.std(dim='run')

DJF_trend_std.min()

: 

In [ ]:
# Multimodel ensemble mean SAT signal-to-noise ratio
# 1958-2022

tas_DJF_ensmean = tas_ano_ds['slope_DJF'].mean(dim='run')
tas_MAM_ensmean = tas_ano_ds['slope_MAM'].mean(dim='run')
tas_JJA_ensmean = tas_ano_ds['slope_JJA'].mean(dim='run')
tas_SON_ensmean = tas_ano_ds['slope_SON'].mean(dim='run')

tas_DJF_emergence = tas_DJF_ensmean/DJF_trend_std
tas_MAM_emergence = tas_MAM_ensmean/MAM_trend_std
tas_JJA_emergence = tas_JJA_ensmean/JJA_trend_std
tas_SON_emergence = tas_SON_ensmean/SON_trend_std

tas_DJF_emergence, display(tas_MAM_emergence), display(tas_JJA_emergence), display(tas_SON_emergence)

: 

In [ ]:
tas_DJF_emergence.min(), tas_DJF_emergence.max()

: 

## Plot the Seasonal Signal-to-noise

In [1]:
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib.colors import BoundaryNorm, ListedColormap

levels = np.arange(0.0, 10.0, 1.0)
# Set up figure
fig = plt.figure(figsize=(12, 8))
gs = gridspec.GridSpec(2, 2, height_ratios=[1, 1], width_ratios=[1, 1], wspace=0.1, hspace=0.1)

# Set up axes
ax1 = plt.subplot(gs[0, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax2 = plt.subplot(gs[0, 1], projection=ccrs.PlateCarree(central_longitude=180))
ax3 = plt.subplot(gs[1, 0], projection=ccrs.PlateCarree(central_longitude=180))
ax4 = plt.subplot(gs[1, 1], projection=ccrs.PlateCarree(central_longitude=180))

ax1.set_extent([0, 360, 0, 90], crs=ccrs.PlateCarree())
ax2.set_extent([0, 360, 0, 90], crs=ccrs.PlateCarree())
ax3.set_extent([0, 360, 0, 90], crs=ccrs.PlateCarree())
ax4.set_extent([0, 360, 0, 90], crs=ccrs.PlateCarree())

# Set up titles
titles = ['a) DJF', 'b) MAM', 'c) JJA', 'd) SON']

# Set up gridlines
gl = ax1.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlocator = mticker.FixedLocator([0, 60, 120, 180, 240, 300, 360])
gl.ylocator = mticker.FixedLocator([0, 30, 60, 90])
gl.xformatter = cticker.LongitudeFormatter()
gl.yformatter = cticker.LatitudeFormatter()
gl.xlabel_style = {'size': 12}
gl.ylabel_style = {'size': 12}

# Set up map features
ax1.coastlines()
ax1.add_feature(cfeature.LAND, facecolor='lightgray')
ax1.add_feature(cfeature.OCEAN, facecolor='white')
ax1.set_title(season, fontsize=10, fontweight='medium')

ax2.coastlines()
ax2.add_feature(cfeature.LAND, facecolor='lightgray')
ax2.add_feature(cfeature.OCEAN, facecolor='white')
ax2.set_title(season, fontsize=10, fontweight='medium')

ax3.coastlines()
ax3.add_feature(cfeature.LAND, facecolor='lightgray')
ax3.add_feature(cfeature.OCEAN, facecolor='white')
ax3.set_title(season, fontsize=10, fontweight='medium')

ax4.coastlines()
ax4.add_feature(cfeature.LAND, facecolor='lightgray')
ax4.add_feature(cfeature.OCEAN, facecolor='white')
ax4.set_title(season, fontsize=10, fontweight='medium')

# Set up titles
ax1.set_title(titles[0], fontsize=12, weight='bold')
ax2.set_title(titles[1], fontsize=12, weight='bold')
ax3.set_title(titles[2], fontsize=12, weight='bold')
ax4.set_title(titles[3], fontsize=12, weight='bold')

# draw the plot
im = ax1.contourf(tas_ano_ds['lon'], tas_ano_ds['lat'], tas_DJF_emergence, cmap='GnBu', levels=levels, extend='both', transform=ccrs.PlateCarree())
im = ax2.contourf(tas_ano_ds['lon'], tas_ano_ds['lat'], tas_MAM_emergence, cmap='GnBu', levels=levels, extend='both', transform=ccrs.PlateCarree())
im = ax3.contourf(tas_ano_ds['lon'], tas_ano_ds['lat'], tas_JJA_emergence, cmap='GnBu', levels=levels, extend='both', transform=ccrs.PlateCarree())
im = ax4.contourf(tas_ano_ds['lon'], tas_ano_ds['lat'], tas_SON_emergence, cmap='GnBu', levels=levels, extend='both', transform=ccrs.PlateCarree())

# Set up colorbar
cbar_ax = fig.add_axes([0.15, 0.1, 0.7, 0.02])
cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal', extend='both', ticks=[-5.0, -4.0, -3.0, -2.0, -1.0, 0, 1.0, 2.0, 3.0, 4.0, 5.0])
cbar.ax.tick_params(labelsize=12)
cbar.set_label(label='SAT signal-to-noise ratio', size=12, weight='bold')

# Save figure
plt.savefig('snr_map.png', dpi=300, bbox_inches='tight')

# Show figure
plt.show()



OpenBLAS blas_thread_init: pthread_create failed for thread 92 of 128: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 93 of 128: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 94 of 128: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 95 of 128: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 96 of 128: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 max
OpenBLAS blas_thread_init: pthread_create failed for thread 97 of 128: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 1024 current, 1024 ma

KeyboardInterrupt: 